# Best Practices for Prompt Engineering

source: https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api


In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('CHAT_COMPLETION_NAME')

# 1. Use the latest model

Use the latest model for best results.

# 2. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [2]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Summarize the text below as a bullet point list of the most important points. \n\n \
        ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###',}],
        max_tokens=400,)

print(response.choices[0].message.content)

- OpenAI and Microsoft are extending their partnership with a multi-year, multi-billion dollar investment from Microsoft.
- The investment will allow OpenAI to continue independent research and develop AI that is safe, useful, and powerful.
- OpenAI remains a capped-profit company and is governed by the OpenAI non-profit in pursuit of their mission to ensure advanced AI benefits all of humanity.
- The partnership with Microsoft is instrumental to OpenAI's progress, as Microsoft shares their vision and values.


# 3. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

In [3]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a poem about OpenAI.',}],
        max_tokens=400,)

print(response.choices[0].message.content)


In the world of AI, there stands a name
A beacon of knowledge, OpenAI's fame
With algorithms and models, they push the frontier
Unleashing the power of technology, crystal clear

They seek the answers to questions profound
In the realm of data, they are unbound
From language to logic, they pave the way
OpenAI's brilliance shines bright every day

Their neural networks weave a web of insight
Unlocking the mysteries of day and night
With innovation and vision, they continue to soar
OpenAI, a marvel unlike any seen before

Their commitment to progress knows no bounds
In the vast expanse of AI, they are renowned
So let us hail the marvels they create
OpenAI, leading us to an enlightened state


In [4]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a short inspiring poem about OpenAI, \
                focusing on the recent DALL-E product launch in the style of Ernest Hemingway',}],
        max_tokens=400,)

print(response.choices[0].message.content)



The launch of DALL-E, a marvel to behold,
Bringing art and creativity to the world,
In a world of ones and zeroes, it breaks the mold,
Unlocking imagination, its banner unfurled.

OpenAI's vision, bold and unafraid,
Pushing boundaries, where others fear to tread,
Innovation thrives where minds are unswayed,
The future beckons, with DALL-E as its thread.


# 4. Articulate the desired output format through examples (example 1, example 2). 

In [5]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Extract the companyn names then years in the following text below and output start index and end index of each entity.\
                Generate output as {"text": "OpenAI", "start": 28, "end": 34} \
                ###\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                ###\
                ',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Here are the company names and years extracted from the text along with their start and end indices:

1. Company: OpenAI
   Start Index: 28
   End Index: 34

2. Year: 2019
   Start Index: 280
   End Index: 284

3. Year: 2021
   Start Index: 288
   End Index: 292


In [6]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Extract the entities mentioned in the text below. \
                Extract the important entities mentioned in the text below. \
                First extract all company names, then extract all years, \
                then extract specific topics which fit the content and finally extract general overarching themes\n\n \
                Desired format: \
                Company names: <comma_separated_list_of_company_names> \
                Years: -||- \
                Specific topics: -||- \
                General themes: -||- \
                """\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                """\
                ',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Company names: OpenAI, Microsoft
Years: 2019, 2021
Specific topics: Partnership, investment, AI research and development
General themes: Collaboration, advancement of AI technology


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune (To update)

In [7]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, investment, multi-year, multi-billion dollar, independent research, AI, safe, useful, powerful


In [8]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below.\n\n \
                Text: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ###\n\
                Text: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords: language models, text processing, API.\n\n\
                ##W"},
                {"role":"user","content": '\n\
                Text: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords:',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, investment, AI, research, independent, safe, useful, powerful


# 6. Reduce “fluffy” and imprecise descriptions

In [9]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing our latest innovation in car seat comfort and safety. The NextGen Car Seat offers advanced ergonomic design, advanced impact protection, and easy installation. With its adjustable features and high-quality materials, this seat is designed to provide ultimate comfort and safety for every journey.


In [10]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the next evolution in child safety and comfort - the all-new SmartSeat 3000. This innovative car seat takes child safety to the next level with its advanced impact protection system and state-of-the-art materials designed to absorb and disperse energy in the event of a collision. The SmartSeat 3000 also boasts a plush, ergonomic design, providing unrivaled comfort for your little one during long car journeys. Additionally, this next-generation car seat features integrated smart technology, including real-time monitoring of the seat's integrity and the ability to alert the parent or caregiver of any issues via a linked smartphone app. With the SmartSeat 3000, you can have peace of mind knowing that your child is protected and comfortable every time you hit the road.


# 7. Instead of just saying what not to do, say what to do instead

In [11]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I understand the frustration. Have you tried resetting your password or using the "forgot password" feature?


In [12]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I understand you're having trouble logging into your account. I recommend reviewing this help article at www.samplewebsite.com/help/faq for tips on troubleshooting login issues. It should provide you with some steps to resolve this problem. If you continue to experience difficulties, please feel free to reach out again, and I can assist you further.


# 8. Code Generation Specific - Use “leading words” to nudge the model toward a particular pattern

In [13]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers',}],
        max_tokens=400,)

print(response.choices[0].message.content)


Here's a simple Python function to achieve this:

```python
def mile_to_km():
    mile = float(input("Enter a number in miles: "))
    km = mile * 1.60934
    print(f"{mile} miles is equal to {km} kilometers.")

mile_to_km()
```

When you run this function, it will prompt you to enter a number in miles, and then it will display the equivalent distance in kilometers.


In [14]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers\n\
                 import ',}],
        max_tokens=400,)

print(response.choices[0].message.content)


Sure! Here's a simple Python function that does that for you:

```python
def convert_miles_to_kilometers():
    miles = float(input("Enter the number of miles: "))
    kilometers = miles * 1.60934
    print(f"{miles} miles is equal to {kilometers} kilometers.")

convert_miles_to_kilometers()
```

You can copy this code into a Python file and run it to convert miles to kilometers.
